In [2]:
import pandas as pd
from unidecode import unidecode
import os
import re

In [3]:
class ProcessadorDeDados():
    def __init__(self, diretorio):
        self.diretorio = diretorio
        self.dataframes = {}
    
    def ler_arquivos(self, arquivos_csv):
        for arquivo in arquivos_csv:
            caminho_arquivo = os.path.join(self.diretorio, arquivo)
            
            if os.path.exists(caminho_arquivo):
                df = pd.read_csv(caminho_arquivo)
                self.dataframes[arquivo] = df
            else:
                print(f'O arquivo {arquivo} não foi encontrado na pasta {self.diretorio}')

    def padronizar_bairro(self, df):
        def padronizacao(texto):
            return unidecode(texto).upper()
        df['BAIRRO'] = df['BAIRRO'].apply(padronizacao)
    
    def renomear_colunas(self, df):
        nomes_colunas = {
            'id': 'id_escola',
            'Escolas_Postos': 'nome_escola',
            'tipo_escola': 'tipo_escola',
            'BAIRRO': 'bairro',
            'ENDEREÇO ': 'endereco',
            'lat': 'latitude',
            'lon': 'longitude',
            'Quantidade': 'qtd_mat'
        }
        df.rename(columns=nomes_colunas, inplace=True)
    
    def remover_acentos_maiusculas(self, df):
        def remover_acentos(texto):
            if isinstance(texto, str):  # Verificar se é uma string
                return unidecode(texto).upper()
            return texto  # Retorna o valor original se não for uma string
        
        df['nome_escola'] = df['nome_escola'].apply(remover_acentos)
        df['endereco'] = df['endereco'].apply(remover_acentos)
        df['bairro'] = df['bairro'].apply(remover_acentos)
        df['subprefeitura'] = df['subprefeitura'].apply(remover_acentos)
    
    def substituir_abreviacoes(self, df):
        mapeamento_abreviacoes = {
            'AV.': 'AVENIDA',
            'R.': 'RUA',
            'EST.': 'ESTRADA',
            'PCA.': 'PRACA',
            'CENTRO INTEGRADO DE EDUCACAO PUBLICA': 'CIEP',
            'ESCOLA MUNICIPAL': 'EM'
        }
        
        def substituir_abreviacoes_em_texto(texto):
            if isinstance(texto, str):  # Verificar se é uma string
                for abreviacao, expandido in mapeamento_abreviacoes.items():
                    texto = texto.replace(abreviacao, expandido)
            return texto

        df['endereco'] = df['endereco'].apply(substituir_abreviacoes_em_texto)
        df['nome_escola'] = df['nome_escola'].apply(substituir_abreviacoes_em_texto)
    
    def novo_tipo_escola(self, df):
        valores_padrao = r'(EM|CIEP|COLEGIO|E.M)'
        posicao_coluna = df.columns.get_loc('nome_escola') + 1
        df.insert(posicao_coluna, 'tipo_escola', '')
        df['tipo_escola'] = df['nome_escola'].str.extract(valores_padrao)
    
    def novo_endereco_numero(self, df):
        def extrair_numero(endereco):
            numeros = re.findall(r'\d+', endereco)
            if numeros:
                return numeros[0]
            else:
                return '000'
        posicao_coluna = df.columns.get_loc('endereco') + 1
        df.insert(posicao_coluna, 'numero', '')
        df['numero'] = df['endereco'].apply(extrair_numero)
        
    def numeros_qtd_material(self, df):
        def verificar_numero(qtd_mat):
            if isinstance(qtd_mat, str):
                numeros = re.findall(r'\d+', qtd_mat)
                if numeros:
                    return numeros[0]
            return 0  # Retorna 0 para valores que não são strings ou não contêm números

        df['qtd_mat'] = df['qtd_mat'].apply(verificar_numero)
    
    def arredondar_coordenadas(self, df):
        df['latitude'] = df['latitude'].apply(lambda x: float(str(x)[:9].replace(",", ".")))
        df['longitude'] = df['longitude'].apply(lambda x: float(str(x)[:9].replace(",", ".")))
    
    def remove_dados(self, df):
        df['endereco'] = df['endereco'].str.replace(r'\d+', '', regex=True, n=1)
        df['nome_escola'] = df['nome_escola'].str.replace(r'EM|CIEP|E.M', '', regex=True)
        df['nome_escola'] = df['nome_escola'].str.replace('[.,]', '', regex=True)
        df['tipo_escola'] = df['tipo_escola'].str.replace('[.,]', '', regex=True)
        df['endereco'] = df['endereco'].str.replace('[.,-]', '', regex=True).str.replace('S/NO', '', regex=True).str.replace('S/N', '', regex=True)

    def organizar_colunas(self, df):
        pop_endereco = df.pop('endereco')
        df.insert(3, 'endereco', pop_endereco)
    
    def processar_dados(self):
        if 'escolas.csv' in self.dataframes and 'material_didatico.csv' and 'subprefeituras.csv' in self.dataframes:
            df_projeto = self.dataframes['escolas.csv'].merge(self.dataframes['material_didatico.csv'], on='id')
            self.padronizar_bairro(self.dataframes['subprefeituras.csv'])
            self.padronizar_bairro(df_projeto)
            df_projeto_final = df_projeto.merge(self.dataframes['subprefeituras.csv'], on='BAIRRO')
            
            self.renomear_colunas(df_projeto_final)
            self.remover_acentos_maiusculas(df_projeto_final)
            self.substituir_abreviacoes(df_projeto_final)
            self.arredondar_coordenadas(df_projeto_final)
            self.novo_tipo_escola(df_projeto_final)
            self.organizar_colunas(df_projeto_final)
            self.novo_endereco_numero(df_projeto_final)
            self.remove_dados(df_projeto_final)
            
            self.numeros_qtd_material(df_projeto_final)

            df_projeto_final = df_projeto_final.drop_duplicates(subset=['nome_escola', 'endereco', 'numero'])
            return df_projeto_final
        else:
            print("Não foi possível encontrar os dataframes necessários.")


In [6]:
if __name__ == "__main__":
    diretorio = 'arquivos/'
    arquivos_csv = ['escolas.csv', 'subprefeituras.csv', 'material_didatico.csv']
    
    processador = ProcessadorDeDados(diretorio)
    processador.ler_arquivos(arquivos_csv)
    dados_processados = processador.processar_dados()
    
    # Exibir o dataframe final
    display(dados_processados.head(50))

,id_escola,nome_escola,tipo_escola,endereco,numero,bairro,latitude,longitude,qtd_mat,subprefeitura
0,178,HENFIL,CIEP,RUA CARLOS SEIDL,000,CAJU,-22.88088,-43.22532,20,CENTRO
1,634,ALICE DO AMARAL PEIXOTO,EM,RUA EBANO,187,BENFICA,-22.88957,-43.23620,121,CENTRO
2,600,URUGUAI,EM,RUA ANA NERI,192,BENFICA,-22.89848,-43.23775,591,CENTRO
3,483,CELESTINO SILVA,EM,RUA DO LAVRADIO,56,CENTRO,-22.90929,-43.18357,220,CENTRO
4,490,TIA CIATA,EM,AVENIDA PRESIDENTE VARGAS,000,CENTRO,-22.90712,-43.19506,578,CENTRO
...,...,...,...,...,...,...,...,...,...,...
138,289,BERTHA LUTZ,EM,ESTRADA DO PIAI,2075,GUARATIBA,-22.97906,-43.67058,329,ZONA OESTE
139,474,ENGENHEIRO GASTAO RANGEL,EM,ESTRADA DO MAGARCA 183,9,GUARATIBA,-22.98046,-43.64354,320,ZONA OESTE
140,301,JONATAS SERRANO,EM,ESTRADA DO MATO ALTO,000,GUARATIBA,-22.95316,-43.57740,335,ZONA OESTE
141,215,NARCISA AMALIA,EM,ESTRADA TEODORETO DE CAMARGO,000,ILHA DE GUARATIBA,-23.00908,-43.53758,0,ZONA OESTE


In [5]:
dados_processados['qtd_mat'] = dados_processados['qtd_mat'].astype(int)
dados_por_subprefeitura = dados_processados.groupby('subprefeitura')['qtd_mat'].sum()

display(dados_por_subprefeitura)

subprefeitura
BARRA DA TIJUCA      215
CENTRO              2352
GRANDE BANGU        3979
GRANDE TIJUCA       2125
ILHAS                741
JACAREPAGUA         2427
ZONA NORTE         13485
ZONA OESTE          8453
ZONA SUL            4256
Name: qtd_mat, dtype: int32

In [192]:
def salvando_dataframe():
    dados_processados.to_csv('./arquivos/dataframe_final.csv')

salvando_dataframe()

In [193]:
def salvando_dados_por_prefeitura():
    dados_por_subprefeitura.to_csv('./arquivos/dados_por_prefeitura.csv')
    
salvando_dados_por_prefeitura()